## Distibuted Training With Hyperparameter tuning using Hyperopt and MLflow

#### Import libs

In [1]:
import numpy as np
import tensorflow as tf
import mlflow
from mlflow.models import infer_signature
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.18.0


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("distibuted_trainingAnd_params_tuning").master("spark://spark-master:7077") \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/31 17:12:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Load Dataset

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = np.array(x_train)

print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)


#### Define model method

In [4]:
# import libs for Tuning
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

# for distibuted training
from hyperopt import SparkTrials

In [5]:
def create_model(l1_noNode,l1_activation,l1_droupout):
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(l1_noNode, activation=l1_activation),
    tf.keras.layers.Dropout(l1_droupout),
    tf.keras.layers.Dense(10)
    ])
    return model

In [6]:
def train_model(p_epoch=2 ,p_optimizer="adam" ,
                l1_noNode=32,l1_activation="relu",l1_droupout=0.2):


    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model = create_model(l1_noNode,l1_activation,l1_droupout)


    model.compile(optimizer=p_optimizer,
              loss=loss_fn,
              metrics=['accuracy'])
    

  
    model.fit(x_train, y_train, epochs=p_epoch)
    eval_loss, eval_acc  = model.evaluate(x_test,  y_test, verbose=2)

    print("eval_loss, eval_acc : ",eval_loss, eval_acc)
    return model, eval_loss, eval_acc




In [7]:
# train_model()

In [8]:
def train_with_hyperopt(params):
    with mlflow.start_run(nested=True) as run:
        p_epoch = params["epochs"]
        p_optimizer = params["optimizer"]
        l1_noNode = params["l1_noNode"]
        l1_activation= params["l1_activation"]
        l1_droupout = params["l1_droupout"]

        model, eval_loss, eval_acc = train_model(p_epoch,p_optimizer,l1_noNode,l1_activation,l1_droupout)



        best_model, eval_loss, eval_acc = train_model(p_epoch=p_epoch,p_optimizer=p_optimizer,
                                                l1_activation=l1_activation,
                                                l1_droupout=l1_droupout,
                                                l1_noNode=l1_noNode)


        mlflow.log_metric("eval_loss",eval_loss)
        mlflow.log_metric("eval_acc",eval_acc)
        model_signature = infer_signature(x_train,best_model.predict(x_train))
        
        mlflow.keras.log_model(best_model,"ths_tune_model",signature=model_signature)

    return {"loss": eval_loss, "status": STATUS_OK, "model": model, "eval_acc": eval_acc}


In [9]:
epochs = [1,2,3,4]
optimizer = ["Adam","SGD","RMSprop"]
l1_noNode = [32,64,128] 
l1_activation = ["relu","softmax","tanh"]
l1_droupout = [0.2,0.3,0.4,0.5]

search_params_space ={
    "epochs": hp.choice("epochs",epochs),
    "optimizer" : hp.choice("optimizer",optimizer),
    "l1_noNode" : hp.choice("l1_noNode",l1_noNode ),
    "l1_activation" : hp.choice("l1_activation",l1_activation),
    "l1_droupout": hp.choice("l1_droupout", l1_droupout)

}


spark_trials = SparkTrials()
algo = tpe.suggest
print(algo)

Because the requested parallelism was None or a non-positive value, parallelism will be set to (2), which is Spark's default parallelism (2), or 1, whichever is greater. We recommend setting parallelism explicitly to a positive value because the total of Spark task slots is subject to cluster sizing.


<function suggest at 0xffff899562a0>


#### Define MLflow experiment

In [10]:
import mlflow
import time
mlflow_uri = "http://mlflow-server:8888/"
mlflow.set_tracking_uri(mlflow_uri)
time.sleep(5)
mlflow.set_experiment("distibuted_trainingAnd_params_tuning_with_mlflow_1")


<Experiment: artifact_location='mlflow-artifacts:/728929677351795026', creation_time=1730389729654, experiment_id='728929677351795026', last_update_time=1730389729654, lifecycle_stage='active', name='distibuted_trainingAnd_params_tuning_with_mlflow_1', tags={}>

In [11]:

import mlflow.pyspark.ml
mlflow.pyspark.ml.autolog()

In [12]:
import pyspark
pyspark.__version__

'3.5.3'

In [13]:

mlflow.tensorflow.autolog()
mlflow.enable_system_metrics_logging()
time.sleep(5)

2024/10/31 17:12:06 WARNING mlflow.utils.autologging_utils: MLflow tensorflow autologging is known to be compatible with 2.7.4 <= tensorflow <= 2.17.0, but the installed version is 2.18.0. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a compatible version, or try upgrading MLflow.


In [14]:
with mlflow.start_run() as run:

    argmin = fmin(
    fn= train_with_hyperopt,
    space=search_params_space,
    algo=algo,
    max_evals = 3,
    # for distrubuted training
    trials=spark_trials 
    )
   


2024/10/31 17:12:11 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2024/10/31 17:12:11 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics becau

  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

2024/10/31 17:12:11 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..

2024/10/31 17:12:11 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.

/usr/local/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



   1/1875 ━━━━━━━━━━━━━━━━━━━━ 4:11 134ms/step - accuracy: 0.0625 - loss: 2.3106
  86/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 594us/step - accuracy: 0.0823 - loss: 2.3049  
 185/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step - accuracy: 0.0861 - loss: 2.3044
 286/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 530us/step - accuracy: 0.0897 - loss: 2.3041
 383/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step - accuracy: 0.0935 - loss: 2.3037
 483/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 522us/step - accuracy: 0.0973 - loss: 2.3035
 583/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 519us/step - accuracy: 0.1003 - loss: 2.3033
 684/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - accuracy: 0.1029 - loss: 2.3031
 783/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - accuracy: 0.1050 - loss: 2.3029
 880/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 515us/step - accuracy: 0.1067 - loss: 2.3027
 981/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 513us/step - accuracy: 0.1082 - loss: 2.3025
1082/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 512us/step - accuracy: 0.1097 - loss: 2.3023
1183/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 511us/step - a

2024/10/31 17:12:13 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.



...

2024/10/31 17:12:53 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.



1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 755us/step - accuracy: 0.5271 - loss: 1.3199

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step         
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step         

313/313 - 0s - 625us/step - accuracy: 0.9222 - loss: 0.6762                     

eval_loss, eval_acc :                                                           
0.6761881709098816                                                              
0.9222000241279602                                                              
   1/1875 ━━━━━━━━━━━━━━━━━━━━ 29s 16ms/step           
   2/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step             
   3/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step             
 112/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step 
 113/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step 
 114/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step 
 115/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step 
 231/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 460us/step 
 232/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step 
 233/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step 
 355/1875 

2024/10/31 17:13:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run entertaining-bat-608 at: http://mlflow-server:8888/#/experiments/728929677351795026/runs/910c4582099147fc8be5f15536ecbd89.

2024/10/31 17:13:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-server:8888/#/experiments/728929677351795026.

2024/10/31 17:13:00 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...

2024/10/31 17:13:00 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!



100%|██████████| 3/3 [00:48<00:00, 16.10s/trial, best loss: 0.33554935455322266]

2024/10/31 17:13:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run mercurial-jay-210 at: http://mlflow-server:8888/#/experiments/728929677351795026/runs/544c41c0cd974472acb45d6ca1327f01.
2024/10/31 17:13:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-server:8888/#/experiments/728929677351795026.
2024/10/31 17:13:00 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/10/31 17:13:00 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [18]:
spark.stop()

### Load trained tf model with Spark DataFrame

#### Set MLflow backend URI

In [16]:
import mlflow
import time
mlflow_uri = "http://mlflow-server:8888/"
mlflow.set_tracking_uri(mlflow_uri)
time.sleep(5)
mlflow.set_experiment("distibuted_trainingAnd_params_tuning_with_mlflow_1")


<Experiment: artifact_location='mlflow-artifacts:/728929677351795026', creation_time=1730389729654, experiment_id='728929677351795026', last_update_time=1730389729654, lifecycle_stage='active', name='distibuted_trainingAnd_params_tuning_with_mlflow_1', tags={}>

#### Initialize SparkSession

In [17]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("distibuted_trainingAnd_params_tuning").master("spark://spark-master:7077") \
        .getOrCreate()

In [18]:
from pyspark.sql.functions import struct, col
logged_model = 'runs:/9794c5f6d6dc41bf9ae605e2abf80364/ths_tune_model'

# Load model as a Spark UDF. Override result_type if the model does not return double values.
loaded_model =  mlflow.pyfunc.load_model(logged_model)


/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

In [19]:
type(x_test)

numpy.ndarray

#### Load dataset

In [20]:
import pandas as pd
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(x_train.shape)

print(x_test.shape)


(60000, 28, 28)
(10000, 28, 28)


In [21]:
pred_ = loaded_model.predict(x_test)
print(pred_)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 394us/step
[[ -8.664396  -14.909999   -3.0911999 ...   8.796654   -8.077242
   -6.6865964]
 [ -6.3493648 -11.766148    9.677436  ... -26.71225    -6.804982
  -25.417377 ]
 [ -9.701519    4.1610136  -2.233385  ...  -2.4515321  -3.1858158
   -6.341231 ]
 ...
 [-18.085354  -16.34156    -7.866031  ...  -2.9642308  -5.7162447
   -1.7017349]
 [ -6.84557   -10.090346   -9.494044  ... -12.040836   -1.1177579
  -11.440213 ]
 [ -5.868529  -17.192024   -4.854329  ... -22.456984  -11.155085
  -16.662743 ]]


In [22]:
# Stop the SparkSession
spark.stop()

In [23]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target
print(X.shape)

(150, 4)
